# Battle of the Neighborhoods
## Venue recommendation by Subway Station in Montreal

### Table of Contents

##### Setup:
    A. Problem
    B. Background
    C. Data
##### Report:
    1. Introduction
    2. Data
    3. Methodology
    4. Results
    5. Discussion
    6. Conclusion
##### Appendix:
    a. Code
    b. Data Sets
    c. Miscelaneous

## **Setup**

### **A. Problem:**

Every summer, the influx of tourists for the festival season creates and incredible gridlock in the streets of Montreal. Much to the dismay of locals, very few people use the amazing public transport system to get from place to place, opting instead to drive around. I believe this is due to the lack of awareness of all the amazing restaurants, bars, theatres, and other points of interest that can easily be reached by subway in a matter of minutes.

### **B. Background:**

As a Montreal local, I have seen it every summer. Droves of tourists clutter the streets of the city, as they try to drive a few blocks in rush-hour traffic. It always boggles the mind that they would opt to spend 20 minutes in downtown traffic, instead of jumping on the subway for a few stops and getting to their destination in relative tranquility (the odd tipsy university student notwithstanding). Figuring it’s probably due to their lack of knowledge, I have decided to help them out by clustering and comparing the top venues near each of our subway stations. 

### **C. Data:**

We will be working mainly with two datasets for this project. 

First and foremost, we need geolocation coordinates for Montreal’s 68 subway (or Metro) stations. The source for these coordinates is the City of Montreal’s Open Data Portal (http://donnees.ville.montreal.qc.ca/dataset). More specifically we will be using their data set on “STM Bus and Subway lines (http://donnees.ville.montreal.qc.ca/dataset/stm-traces-des-lignes-de-bus-et-de-metro). Now unfortunately, they only make it available as a large .SHP Shapefile, so we are gong to have to do a lot of cleanup to make it workable.

Our second dataset will be the venues information queried from Foursquare using the geolocation coordinates obtained above. Instead of focusing on quantity (i.e. concentration of venues in a location), we will be focusing on quality (i.e. what are the top venues in a location). We are, after all, trying to convince tourists to use our world-class public transport system instead of contributing to the summer gridlock – and what better way than to guide them to the best Metro stations, with the best venues?


## **Appendix**

### **Code**

#### Start by importing the libraries we need for this project.

In [1]:
import pandas as pd
import geopandas as gpd
!pip install matplotlib -U
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import folium
from folium import plugins
import seaborn as sns
!pip install descartes -U
import descartes
import re
import requests

Requirement already up-to-date: matplotlib in c:\users\alexi\anaconda3\lib\site-packages (3.2.1)



Bad key "text.kerning_factor" on line 4 in
C:\Users\alexi\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


Requirement already up-to-date: descartes in c:\users\alexi\anaconda3\lib\site-packages (1.1.0)


#### Because we're working with a Shapefile, we are using a geoPandas geoDataFrame. We will eventually revert back to a Pandas DataFrame for convenience's sake.

In [2]:
df = gpd.read_file('stm_arrets_sig.shp')
df.head()

,stop_id,stop_code,stop_name,stop_url,wheelchair,route_id,loc_type,service_id,geometry
0,43-01,10118,Station Angrignon,None,2,None,2,20M,POINT (296677.562 5034048.338)
1,43,10118,Station Angrignon,http://www.stm.info/fr/infos/reseaux/metro/ang...,2,1,0,20M,POINT (296733.669 5034064.602)
2,42-01,10120,Station Monk - Édicule Nord,None,2,None,2,20M,POINT (297515.753 5034601.626)
3,42-02,10120,Station Monk - Édicule Sud,None,2,None,2,20M,POINT (297496.004 5034568.310)
4,42,10120,Station Monk,http://www.stm.info/fr/infos/reseaux/metro/monk,2,1,0,20M,POINT (297506.817 5034585.078)


#### Let's do a little bit of cleanup. Having had a look-through the file, I noticed we could drop all of the rows with "None" for route_id as they are all duplicates. We're also going to drop some useless columns.

In [3]:
df.replace(r'None', np.nan, regex=True, inplace = True)
df.dropna(axis = 0, how = "any", inplace = True)
df.drop(['stop_id', 'stop_code', 'wheelchair', 'loc_type', 'service_id'], axis = 1, inplace = True)

#### Every subway station, and bus stop has an associated URL (it's used to look up subway and bus schedules). Let's create a new dataframe containing only those entries where the URL contains the word "metro" (as this indicates this is a subway station).

In [4]:
df_metro = df[df['stop_url'].str.contains('.*metro.*')]

#### Let's re-project the geometry data to a coordinate system we are more familiar with (and something that folium will work with without complaining too much).

In [5]:
df_metro = df_metro.to_crs(epsg='4326')

#### Now let's convert the geoDataFrame to a regular DataFrame, and cast the "geometry" column to string so that we may parse it with regex. We have to do this because we started with a geoDataFrame created from a shapefile.

In [6]:
df_metro = pd.DataFrame(df_metro)
df_metro['geometry'] = df_metro['geometry'].astype('str')
print(df_metro.dtypes)
df_metro.head()

stop_name    object
stop_url     object
route_id     object
geometry     object
dtype: object


,stop_name,stop_url,route_id,geometry
1,Station Angrignon,http://www.stm.info/fr/infos/reseaux/metro/ang...,1,POINT (-73.60311799999998 45.44646599999288)
4,Station Monk,http://www.stm.info/fr/infos/reseaux/metro/monk,1,POINT (-73.593242 45.45115799999289)
6,Station Jolicoeur,http://www.stm.info/fr/infos/reseaux/metro/jol...,1,POINT (-73.58169099999999 45.45700999999288)
9,Station Verdun,http://www.stm.info/fr/infos/reseaux/metro/verdun,1,POINT (-73.57202099999999 45.45944099999288)
12,Station De l'Église,http://www.stm.info/fr/infos/reseaux/metro/de-...,1,POINT (-73.56707400000001 45.46189399999288)


Now, let's parse the clunky database, and extract all of the useful information into a new dataframe. We will take this opportunity to clean up the geolocation coordinates and make something more usable.

In [7]:
df_metro_geo = pd.DataFrame()
df_metro_geo['stop'] = ''
df_metro_geo['lat'] = ''
df_metro_geo['lon'] = ''

In [8]:
for name, geometry in zip('df_metro.stop_name', 'df_metro.geometry'):
    df_metro_geo.stop = df_metro.stop_name
    df_metro_geo.lon = df_metro.geometry.str.extract(pat = r"(-[0-9][0-9].[0-9]*)")
    df_metro_geo.lat = df_metro.geometry.str.extract(pat = r"[0-9]\s([0-9][0-9].[0-9]*)")

In [9]:
df_metro_geo.head()

,stop,lat,lon
1,Station Angrignon,45.44646599999288,-73.60311799999998
4,Station Monk,45.45115799999289,-73.593242
6,Station Jolicoeur,45.45700999999288,-73.58169099999999
9,Station Verdun,45.45944099999288,-73.57202099999999
12,Station De l'Église,45.46189399999288,-73.56707400000001


#### Finally, let's draw a map of Montreal, and use our newly cleaned up geolocation coordinates to mark all of the subway stations.

In [10]:
mtl_map = folium.Map(location = [45.52, -73.62], zoom_start = 12, tiles = 'stamenterrain')

for row in df_metro_geo.itertuples():
    mtl_map.add_child(folium.CircleMarker(location = [row.lat, row.lon],
                                         radius = 5,
                                         fill = True,
                                         fill_color = 'red',
                                         fill_opacity = 0.7,
                                         popup = row.stop))

mtl_map

#### Ok, now let's make some queries to Foursquare using our subway stations coordinates. Couple of things to note - we're going to limit the number of venues to 10 per location, and have Foursquare return only the 5 most popular venues (passing "sortByPopularity = 1, and topPicks).

In [11]:
import config as cfg

CLIENT_ID = cfg.client_id
CLIENT_SECRET = cfg.client_secret
VERSION = "20200426"

In [12]:
radius = 500
LIMIT = 5

venues = []

for stop, lat, lon in zip(df_metro_geo['stop'], df_metro_geo['lat'], df_metro_geo['lon']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&sortByPopularity=1&section=topPicks".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lon,
        radius, 
        LIMIT)

    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            stop,
            lat, 
            lon, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

#### Dump the returned venues into a Pandas DataFrame, & add some column headers. Note that some locations have less than 5 venues. That's normal as these are periphery stations where there's little other than the subway station and a park.

In [13]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['stop', 'Lat', 'Lon', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

venues_df.head(40)

,stop,Lat,Lon,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Station Angrignon,45.44646599999288,-73.60311799999998,Parc Angrignon,45.443001,-73.603334,Park
1,Station Angrignon,45.44646599999288,-73.60311799999998,Fort Angrignon,45.444657,-73.602031,Playground
2,Station Monk,45.45115799999289,-73.593242,Dilallo Burger,45.450364,-73.598175,Deli / Bodega
3,Station Monk,45.45115799999289,-73.593242,Tabagie monk,45.451193,-73.593383,Smoke Shop
4,Station Monk,45.45115799999289,-73.593242,Fruiterie Shaana,45.452895,-73.594017,Grocery Store
5,Station Monk,45.45115799999289,-73.593242,SAQ,45.451420,-73.593369,Liquor Store
6,Station Monk,45.45115799999289,-73.593242,Parc clifford - Verdun,45.450364,-73.589213,Baseball Field
7,Station Jolicoeur,45.45700999999288,-73.58169099999999,Chez Jacquie Et France,45.458544,-73.576102,Breakfast Spot
8,Station Jolicoeur,45.45700999999288,-73.58169099999999,La Terrasse,45.455548,-73.576462,Breakfast Spot
9,Station Jolicoeur,45.45700999999288,-73.58169099999999,Parc de la verendrye,45.456815,-73.582016,Park


#### Get dummy variables.

In [14]:
venues_oh = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

venues_oh.head()

,Art Museum,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Baseball Field,...,Strip Club,Supermarket,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
venues_oh.insert(0, 'stop', venues_df['stop'])

central_venues = venues_oh.groupby(["stop"]).mean().reset_index()

In [16]:
central_venues.head()

,stop,Art Museum,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,...,Strip Club,Supermarket,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Station Acadie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0
1,Station Angrignon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Station Assomption,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Station Atwater,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Station Beaubien,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0


In [17]:
areaColumns = ['stop']
freqColumns = []
for ind in np.arange(5):
    freqColumns.append('Top {}'.format(ind+1))

columns = areaColumns+freqColumns

top5_venues = pd.DataFrame(columns=columns)
top5_venues['stop'] = central_venues['stop']

top5_venues.head()

,stop,Top 1,Top 2,Top 3,Top 4,Top 5
0,Station Acadie,NaN,NaN,NaN,NaN,NaN
1,Station Angrignon,NaN,NaN,NaN,NaN,NaN
2,Station Assomption,NaN,NaN,NaN,NaN,NaN
3,Station Atwater,NaN,NaN,NaN,NaN,NaN
4,Station Beaubien,NaN,NaN,NaN,NaN,NaN


#### Get top 5 venues for each location.

In [18]:
for ind in np.arange(central_venues.shape[0]):
    row_categories = central_venues.iloc[ind, :].iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    top5_venues.iloc[ind, 1:] = row_categories_sorted.index.values[0:5]

In [19]:
top5_venues.head()

,stop,Top 1,Top 2,Top 3,Top 4,Top 5
0,Station Acadie,Convenience Store,Greek Restaurant,Toy / Game Store,Coffee Shop,Sushi Restaurant
1,Station Angrignon,Playground,Park,Gym / Fitness Center,Concert Hall,Convenience Store
2,Station Assomption,Gym / Fitness Center,Greek Restaurant,Restaurant,Beer Store,Breakfast Spot
3,Station Atwater,Grocery Store,Supermarket,Office,Shopping Mall,Movie Theater
4,Station Beaubien,Vietnamese Restaurant,Bakery,Pizza Place,Beer Store,Café


#### Import and run k-means clustering on the venues.

In [20]:
from sklearn.cluster import KMeans

In [21]:
cluster_df = central_venues.drop(["stop"], axis = 1)

kmeans = KMeans(n_clusters=5).fit(cluster_df)

In [22]:
cluster_df = df_metro_geo.copy()
cluster_df = pd.merge(cluster_df, central_venues, on=['stop'], how='inner')
cluster_df.head()

,stop,lat,lon,Art Museum,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,Bagel Shop,Bakery,...,Strip Club,Supermarket,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Station Angrignon,45.44646599999288,-73.60311799999998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Station Monk,45.45115799999289,-73.593242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Station Jolicoeur,45.45700999999288,-73.58169099999999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Station Verdun,45.45944099999288,-73.57202099999999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Station De l'Église,45.46189399999288,-73.56707400000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0


#### Add in the cluster labels at the begining of the dataframe.

In [23]:
cluster_df["Cluster_labels"] = kmeans.labels_
cluster_df = cluster_df.join(top5_venues.set_index("stop"), on="stop")
cluster_df.head()

,stop,lat,lon,Art Museum,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,Bagel Shop,Bakery,...,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio,Cluster_labels,Top 1,Top 2,Top 3,Top 4,Top 5
0,Station Angrignon,45.44646599999288,-73.60311799999998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,Playground,Park,Gym / Fitness Center,Concert Hall,Convenience Store
1,Station Monk,45.45115799999289,-73.593242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,Baseball Field,Grocery Store,Deli / Bodega,Liquor Store,Smoke Shop
2,Station Jolicoeur,45.45700999999288,-73.58169099999999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,Breakfast Spot,Park,Concert Hall,Coworking Space,Creperie
3,Station Verdun,45.45944099999288,-73.57202099999999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,Breakfast Spot,Grocery Store,Cheese Shop,Indian Restaurant,Beer Store
4,Station De l'Église,45.46189399999288,-73.56707400000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2,0.0,0.0,0.0,3,Restaurant,Grocery Store,Trail,Café,Cheese Shop


In [24]:
cluster_df.sort_values(["Cluster_labels"], inplace=True)

mid = cluster_df['Cluster_labels']
cluster_df.drop(labels=['Cluster_labels'], axis=1,inplace = True)
cluster_df.insert(0, 'Cluster_labels', mid)
cluster_df = cluster_df[['Cluster_labels', 'stop', 'lat', 'lon', 'Top 1', 'Top 2', 'Top 3', 'Top 4', 'Top 5']]

cluster_df = cluster_df.reset_index(drop=True)
print(cluster_df.shape)
cluster_df.head()

(72, 9)


,Cluster_labels,stop,lat,lon,Top 1,Top 2,Top 3,Top 4,Top 5
0,0,Station Saint-Michel,45.55955999999278,-73.600002,Grocery Store,Liquor Store,Metro Station,Market,Vegetarian / Vegan Restaurant
1,0,Station Radisson,45.58943099999274,-73.539269,Gastropub,Asian Restaurant,Shopping Mall,Restaurant,Italian Restaurant
2,0,Station Langelier,45.58273599999276,-73.543133,Restaurant,Automotive Shop,Thai Restaurant,Chinese Restaurant,Italian Restaurant
3,0,Station Viau,45.56068699999277,-73.54752999999999,Gym / Fitness Center,Athletics & Sports,Sports Club,Movie Theater,Fast Food Restaurant
4,0,Station Joliette,45.54683199999279,-73.55139099999998,Restaurant,Garden,Park,Baseball Stadium,Yoga Studio


#### Let's define some colors for these clusters.

In [25]:
color_list = cluster_df["Cluster_labels"]
color_df = pd.DataFrame(color_list)
color_df.rename(columns = {'Cluster_labels':'colors'}, inplace = True)

In [26]:
color_df["colors"] = color_df["colors"].replace(0, 'yellow')
color_df["colors"] = color_df["colors"].replace(1, 'red')
color_df["colors"] = color_df["colors"].replace(2, 'blue')
color_df["colors"] = color_df["colors"].replace(3, 'green')
color_df["colors"] = color_df["colors"].replace(4, 'purple')
cluster_df.insert(0, 'colors', color_df)

#### And now, let's have a look at a map of these clusters.

In [27]:
mtl_venue_map = folium.Map(location = [45.52, -73.62], zoom_start = 12, tiles = 'stamenterrain')

for row in cluster_df.itertuples():
    mtl_venue_map.add_child(folium.CircleMarker(location = [row.lat, row.lon],
                                  color = row.colors,
                                  fill = True,
                                  fill_color = row.colors,
                                  fill_opacity = 0.5,
                                  popup = [("stop:", row.stop), ("Cluster:", row.Cluster_labels)]))

mtl_venue_map

In [28]:
cluster_df.loc[cluster_df['Cluster_labels'] == 0, cluster_df.columns[[1] + [2] + list(range(5, cluster_df.shape[1]))]]

,Cluster_labels,stop,Top 1,Top 2,Top 3,Top 4,Top 5
0,0,Station Saint-Michel,Grocery Store,Liquor Store,Metro Station,Market,Vegetarian / Vegan Restaurant
1,0,Station Radisson,Gastropub,Asian Restaurant,Shopping Mall,Restaurant,Italian Restaurant
2,0,Station Langelier,Restaurant,Automotive Shop,Thai Restaurant,Chinese Restaurant,Italian Restaurant
3,0,Station Viau,Gym / Fitness Center,Athletics & Sports,Sports Club,Movie Theater,Fast Food Restaurant
4,0,Station Joliette,Restaurant,Garden,Park,Baseball Stadium,Yoga Studio
5,0,Station Square-VictoriaOACI,Plaza,Building,Coworking Space,Cocktail Bar,Café
6,0,Station Berri-UQAM 2,Coffee Shop,Hostel,Record Shop,Gastropub,Movie Theater
7,0,Station Berri-UQAM 4,Coffee Shop,Hostel,Record Shop,Gastropub,Movie Theater
8,0,Station Jarry,Convenience Store,Grocery Store,Bakery,Discount Store,Café
9,0,Station Henri-Bourassa,Hobby Shop,Coffee Shop,Restaurant,Park,Yoga Studio


In [29]:
cluster_df.loc[cluster_df['Cluster_labels'] == 1, cluster_df.columns[[1] + [2] + list(range(5, cluster_df.shape[1]))]]

,Cluster_labels,stop,Top 1,Top 2,Top 3,Top 4,Top 5
18,1,Station Plamondon,Grocery Store,Indian Restaurant,Bakery,Filipino Restaurant,Chinese Restaurant
19,1,Station D'Iberville,Vietnamese Restaurant,Pet Store,Peruvian Restaurant,Coffee Shop,Discount Store
20,1,Station Place-Saint-Henri,Hawaiian Restaurant,Malay Restaurant,Park,Café,Breakfast Spot
21,1,Station Sauvé,Pizza Place,Pharmacy,Baseball Field,Fast Food Restaurant,Convenience Store
22,1,Station Place-d'Armes,Plaza,Shopping Mall,Sports Bar,Church,Food Court
23,1,Station Champ-de-Mars,Plaza,Pub,Breakfast Spot,Café,Asian Restaurant
24,1,Station Mont-Royal,Restaurant,Bookstore,Bagel Shop,Bakery,Supermarket
25,1,Station Université-de-Montréal,Snack Place,College Cafeteria,College Bookstore,Café,Concert Hall
26,1,Station Namur,Restaurant,Fast Food Restaurant,Movie Theater,Market,Yoga Studio
27,1,Station Rosemont,Café,Bar,Pizza Place,Bookstore,Supermarket


In [30]:
cluster_df.loc[cluster_df['Cluster_labels'] == 2, cluster_df.columns[[1] + [2] + list(range(5, cluster_df.shape[1]))]]

,Cluster_labels,stop,Top 1,Top 2,Top 3,Top 4,Top 5
38,2,Station Côte-Vertu,Flower Shop,Hot Dog Joint,Italian Restaurant,Bank,Convenience Store
39,2,Station Jolicoeur,Breakfast Spot,Park,Concert Hall,Coworking Space,Creperie
40,2,Station Cartier,Hockey Arena,Coffee Shop,Sushi Restaurant,Fast Food Restaurant,Strip Club
41,2,Station Peel,Art Museum,Church,Restaurant,Movie Theater,Bookstore
42,2,Station Beaubien,Vietnamese Restaurant,Bakery,Pizza Place,Beer Store,Café
43,2,Station Saint-Laurent,Concert Hall,Movie Theater,Performing Arts Venue,Plaza,Gourmet Shop
44,2,Station Berri-UQAM 1,Coffee Shop,Hostel,Record Shop,Gastropub,Movie Theater
45,2,Station Sherbrooke,Gym / Fitness Center,French Restaurant,Park,Breakfast Spot,Grocery Store
46,2,Station Snowdon 5,Deli / Bodega,Bakery,Chinese Restaurant,Donut Shop,Bookstore
47,2,Station Bonaventure,Building,Hockey Arena,Church,Movie Theater,Skating Rink


In [31]:
cluster_df.loc[cluster_df['Cluster_labels'] == 3, cluster_df.columns[[1] + [2] + list(range(5, cluster_df.shape[1]))]]

,Cluster_labels,stop,Top 1,Top 2,Top 3,Top 4,Top 5
53,3,Station De l'Église,Restaurant,Grocery Store,Trail,Café,Cheese Shop
54,3,Station Acadie,Convenience Store,Greek Restaurant,Toy / Game Store,Coffee Shop,Sushi Restaurant
55,3,Station Outremont,Bakery,Playground,Park,Spanish Restaurant,Electronics Store
56,3,Station LaSalle,Convenience Store,Gourmet Shop,Italian Restaurant,Restaurant,Cambodian Restaurant
57,3,Station Cadillac,Breakfast Spot,Grocery Store,Restaurant,Italian Restaurant,Park
58,3,Station LongueuilUniversité-de-Sherbrooke,Middle Eastern Restaurant,Poutine Place,Bakery,Pizza Place,Yoga Studio
59,3,Station Assomption,Gym / Fitness Center,Greek Restaurant,Restaurant,Beer Store,Breakfast Spot
60,3,Station Vendôme,Bakery,Greek Restaurant,Coffee Shop,Pub,Sushi Restaurant
61,3,Station Crémazie,Restaurant,Breakfast Spot,Pub,Liquor Store,Fast Food Restaurant
62,3,Station Place-des-Arts,Concert Hall,Plaza,Performing Arts Venue,Pizza Place,Gourmet Shop


In [32]:
cluster_df.loc[cluster_df['Cluster_labels'] == 4, cluster_df.columns[[1] + [2] + list(range(5, cluster_df.shape[1]))]]

,Cluster_labels,stop,Top 1,Top 2,Top 3,Top 4,Top 5
68,4,Station Côte-Sainte-Catherine,Gym,Sri Lankan Restaurant,Chinese Restaurant,Playground,Filipino Restaurant
69,4,Station Côte-des-Neiges,Farmers Market,Pub,Bakery,Russian Restaurant,Bar
70,4,Station Jean-Talon 2,Bakery,Farmers Market,Grocery Store,Thrift / Vintage Store,Dessert Shop
71,4,Station Monk,Baseball Field,Grocery Store,Deli / Bodega,Liquor Store,Smoke Shop
